# The Basics of Merging Dataframes in Pandas

As I continue to learn Python's data science toolkit (which seems as vast and as ever-expanding as the universe, by the way), I've come across some useful information in how to manipulate pandas dataframes, and also some of the basics with generating plots in Matplotlib.  I wrote about that experience [here](https://medium.com/@joelmsherman/adventures-in-python-15fd17ca211e) and [here](https://medium.com/@joelmsherman/basic-data-viz-with-matplotlib-602c885f8093) because the best way to learn is to try to explain things to others, right?  

In this installment of learning by teaching, I will tackle some of the basics of merging dataframes in Pandas. But before I get into it, you may be wondering what I mean by **merging dataframes** and maybe more importantly, why you should care about the topic.

## What is "Merging" and Why Is It Important?

When doing data analysis, more often than not one needs to work with more than one data table or data source.  Kaggle is nice for providing nice clean single csv files for your machine learning project, but this often isn't how we do our work in the real-world.  It's estimated and commonly cited that the bulk (80%) of a data scientist's time is spent data wrangling and cleaning.  Merging comes into place when we need to combine multiple datasets or disparate sources of data into one unified frame for analysis.  

Quite literally, we can combine data vertically (stack two or more tables with similar columns to combine rows) or horizontally (stack two or more tables with similar rows to combine columns), and Pandas offers a variety of tools for doing this, including **append()**, **concat()**, **join()** and, you guessed it, **merge()**.  And when I use the term "merge", I refer to any and all of these methods unless I specifically state "merge method".

## Why Not Do Your Merging In SQL?

I know what you're thinking, and as a long-time SQL developer, let me just say duh!  This concept is so engrained in SQL's DNA that it's hard not to want to prepare or engineer your frame in SQL first, before bringing it into a development environment for further modeling and analysis.  But here's the problem:  not all data worthy of analysis is structured, nor lives in an RMDBMS! More (most?) often, the data we need for machine learning models or data visualizations is ad hoc, highly unstructured and dispersed in a variety of locations.  Once again, the power of Pandas and merging!

I’ll stick with my own personal sleep and training data for this exercise (pun intended) and make the files available on my public github account [here](https://github.com/joelmsherman/explain_dataframe-merging/blob/master/sleep.csv) and [here](https://github.com/joelmsherman/explain_dataframe-merging/blob/master/training_recovery.csv) in case anyone wants to replicate my work here.

### Import Dataframes

In [108]:
# Import Libraries
import pandas as pd
import numpy as np

# Locate Data Sources
url_sleep = 'https://raw.githubusercontent.com/joelmsherman/explain_dataframe-merging/master/sleep.csv'
url_trainrecov = 'https://raw.githubusercontent.com/joelmsherman/explain_dataframe-merging/master/training_recovery.csv'

# Identify Desired Columns
cols_sleep = ['date',
              'Total Sleep Time',
              'Awake Time',
              'REM Sleep Time',
              'Light Sleep Time',
              'Deep Sleep Time',
              'Restless Sleep',
              'Sleep Efficiency',
              'Sleep Latency',
              'Average Resting Heart Rate',
              'Lowest Resting Heart Rate',
              'Average HRV',
              'Temperature Deviation',
              'Respiratory Rate']

cols_trainrecov = ['date',
                   'HRV4T_Recovery_Points',
                   'trainingRPE',
                   'trainingTSS',
                   'alcohol']

# Read the data into Pandas Dataframes
Sleep = pd.read_csv(url_sleep,skipinitialspace=True, usecols=cols_sleep)
TrRec = pd.read_csv(url_trainrecov, skipinitialspace=True, usecols=cols_trainrecov)

### Clean Dataframe 1: Sleep Data

In [109]:
# Clean up and set the index
Sleep['date'] = pd.to_datetime(Sleep['date']).dt.date
Sleep.set_index('date', inplace=True)

# Column labels
Sleep.columns = ['TotalSleep', 'Awake', 'REM', 'Light', 'Deep', 'Restless', 'Efficiency', 'Latency', 'AvgRHR', 'LowRHR', 'AvgHRV', 'TDiff', 'RespR']

# Fill NaN with mean of each column for which the NaN exists
Sleep = Sleep.fillna(Sleep.mean()) 

### Clean Dataframe 2: Training and Recovery Data

In [110]:
# Clean up and set the index
TrRec['date'] = pd.to_datetime(TrRec['date']).dt.date
TrRec.set_index('date', inplace=True)

# Column labels
TrRec.columns = ['RecPts', 'RPE', 'TSS', 'Beers']

# Convert Recovery pts to numeric and assign mean to NaN
TrRec['RecPts'] = pd.to_numeric(TrRec['RecPts'], errors='coerce')
TrRec['RecPts'] = TrRec['RecPts'].fillna((TrRec['RecPts'].mean()))

# Convert RPE to numeric and assign zero to NaN for non-workout days
TrRec['RPE'] = pd.to_numeric(TrRec['RPE'], errors='coerce')
TrRec['RPE'] = TrRec['RPE'].fillna(0)

# Convert TSS to numeric and imput NaN based on values of RPE
TrRec['TSS'] = pd.to_numeric(TrRec['TSS'], errors='coerce')
conditions = [TrRec['RPE']==0, TrRec['RPE'].between(0.1, 3), TrRec['RPE'].between(3.1, 5), TrRec['RPE'].between(5.1, 7), TrRec['RPE'] > 7]
values = [0, 41, 53, 77, 85]
TrRec['TSS'] = np.where(TrRec['TSS'].isnull(),
                              np.select(conditions, values),
                              TrRec['TSS'])

### Merge Dataframes into STR (Sleep, Training and Recovery)

In [132]:
STR = pd.merge(Sleep, TrRec, on='date')
STR.to_csv('STR.csv')

### For Medium Article from Here Down

In [112]:
# Subset Sleep df into two separate frames with 7 records each, and with only 'REM', 'Deep' and 'Light' cols
# Subset TrRec df into a 14 day period (same days above)  and with only 'RecPts', 'RPE' and 'TSS'

[Pick up Article Here] After some subsetting and data cleaning, let's say I have two, one-week spans of sleep data that look like this

In [113]:
# Print Sleep1, Sleep2 frames side by side?

And one, two-week span of training and recovery data that looks like this

In [114]:
# Print TrRec df

How can we bring these dataframes together into one dataframe for analysis?  Let's start with the sleep data, which are weekly files of the same structure.  We obviously need to stack these frames vertically, and we have a couple options.  

### Option 1: Append()

The append method is like a SQL union, and allows dataframes to be merged vertically, on top of each other. We can do this to our sleep dataframes using append() like this

In [115]:
# Demo Append

### Option 2: Concat()

As an alternative to append, concat can merge dataframes either vertically or horizontally, using the axix=1 for the latter, and axis=0 for the former.  Here it is in action for our sleep dataframes 

In [116]:
# Demo Concat

Great!  We've now got a single sleep dataframe with data for a two week period.  Now we just need to keep playing our game of tetris and merge our training and recovery data to this and we're done.  Here again, we've got a couple of options, this time for merging horizontally.

### Option 1: Concat()

### Option 2: Join()

### Option 3: Merge() 